In [76]:
import pandas as pd

In [77]:
input_file = "input.txt"

In [78]:
with open(input_file) as file:
    plant_types = set(file.read().replace("\n", ""))
with open(input_file) as file:
    df = pd.DataFrame(list(map(lambda x: list(x.strip()), file.readlines())))

{'C', 'E', 'F', 'I', 'J', 'M', 'R', 'S', 'V'}

In [79]:
df

,0,1,2,3,4,5,6,7,8,9
0,R,R,R,R,I,I,C,C,F,F
1,R,R,R,R,I,I,C,C,C,F
2,V,V,R,R,R,C,C,F,F,F
3,V,V,R,C,C,C,J,F,F,F
4,V,V,V,V,C,J,J,C,F,E
5,V,V,I,V,C,C,J,J,E,E
6,V,V,I,I,I,C,J,J,E,E
7,M,I,I,I,I,I,J,J,E,E
8,M,I,I,I,S,I,J,E,E,E
9,M,M,M,I,S,S,J,E,E,E


In [80]:
from collections import namedtuple
Direction = namedtuple("Direction", ["vertical", "horizontal"])
Point = namedtuple("Point", ["x", "y"])

def get_positions_of_plant_type(plant_type, field):
    coordinates = field[field == plant_type].stack().index.values
    points = list(map(lambda x: Point(x=x[1], y=x[0]), coordinates))
    return points

def get_neighbour(point: Point, direction: Direction):
    return Point(x= point.x + direction.horizontal, y = point.y + direction.vertical)



def get_region(starting_point, positions):
    all_points = set(positions)
    points_to_discover = [starting_point]
    result = []
    while points_to_discover:
        element = points_to_discover.pop()
        result.append(element)
        neighbours = [get_neighbour(element, Direction(vertical=1, horizontal=0)),
            get_neighbour(element, Direction(vertical=-1, horizontal=0)),
            get_neighbour(element, Direction(vertical=0, horizontal=1)),
            get_neighbour(element, Direction(vertical=0, horizontal=-1))]
        neighbours = list(filter(lambda x: x in all_points, neighbours))
        points_to_discover = points_to_discover + neighbours
        all_points = all_points - set(neighbours)
    return (result, list(all_points))


def get_regions(plant_type, positions):
    regions = []
    available_points = positions
    while available_points:
        starting_point = available_points.pop()
        region, updated_available_points = get_region(starting_point, available_points)
        available_points = updated_available_points
        regions.append(region)
    return regions

def get_fences(region):
    region_points = set(region)
    fences_per_point = {}
    for point in region:
        neighbours = set([get_neighbour(point, Direction(vertical=1, horizontal=0)),
            get_neighbour(point, Direction(vertical=-1, horizontal=0)),
            get_neighbour(point, Direction(vertical=0, horizontal=1)),
            get_neighbour(point, Direction(vertical=0, horizontal=-1))])
        outside_neighbours = neighbours - region_points
        fences_per_point[point] = len(outside_neighbours)
    return fences_per_point

def get_sides(region):
    region_points = set(region)
    fences = {}
    for point in region:
        directions = set([Direction(vertical=1, horizontal=0),
            Direction(vertical=-1, horizontal=0),
            Direction(vertical=0, horizontal=1),
            Direction(vertical=0, horizontal=-1)])
        
        for direction in directions:
            neighbour = get_neighbour(point, direction)
            if neighbour in region_points:
                continue
            relevant_axis = point.y if direction.vertical != 0 else point.x
            other_axis = point.x if direction.vertical != 0 else point.y
            side_key = (relevant_axis, direction)
            if side_key in fences:
                fences[side_key].append(other_axis)
            else:
                fences[side_key] = [other_axis]

    for side_key in fences.keys():
        num_of_sides = get_number_of_uninterrupted_series(fences[side_key])
        fences[side_key] = num_of_sides
            
    return fences


def get_number_of_uninterrupted_series(some_int_list):
    some_int_list.sort()
    num_of_series = 1
    for index, value in enumerate(some_int_list):
        if index >= len(some_int_list)-1:
            continue
        next_value = some_int_list[index+1]
        if abs(next_value-value) > 1:
            num_of_series += 1
    return num_of_series


result1 = 0
result2 = 0
for plant_type in list(plant_types):

    plant_positions = get_positions_of_plant_type(plant_type, df)
    regions = get_regions(plant_type, plant_positions)
    for region in regions:
        fences = get_fences(region)
        sides = get_sides(region)


        number_of_fences = sum(fences.values())
        number_of_sides = sum(sides.values())

        result1 += (number_of_fences*len(region))
        result2 += (number_of_sides*len(region))
result1, result2
    

(1930, 1206)